In [35]:
import tfim
import tfim_perturbation
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy import linalg
import matplotlib.pyplot as plt
from scipy import optimize
import progressbar

In [36]:
# Initial system specification
L = [4]
Jij_seed = 7
h_x_range = np.arange(0, 0.1, 0.001)

In [37]:
PBC = True
J = 1

In [38]:
# Build lattice and basis
###################################
lattice = tfim.Lattice(L, PBC)
N = lattice.N
basis = tfim.IsingBasis(lattice)
###################################

#construct random J matrix
Jij = tfim.Jij_instance(N,J,"bimodal",7) 

In [39]:
# List out all the spin_states, corresponding indices and energies
Energies = -tfim.JZZ_SK_ME(basis,Jij)
for index in range(2**N):
    print index, basis.state(index), Energies[index]

0 [0 0 0 0] -0.0
1 [0 0 0 1] 2.0
2 [0 0 1 0] -6.0
3 [0 0 1 1] -0.0
4 [0 1 0 0] 2.0
5 [0 1 0 1] -0.0
6 [0 1 1 0] -0.0
7 [0 1 1 1] 2.0
8 [1 0 0 0] 2.0
9 [1 0 0 1] -0.0
10 [1 0 1 0] -0.0
11 [1 0 1 1] 2.0
12 [1 1 0 0] -0.0
13 [1 1 0 1] -6.0
14 [1 1 1 0] 2.0
15 [1 1 1 1] -0.0


In [40]:
# Build 2nd order approximated matrix

GS_energy, GS_indices = tfim_perturbation.GS(Energies)

H_0 = tfim_perturbation.H_0(GS_energy, GS_indices)

H_app_1 = tfim_perturbation.H_app_1(basis, GS_indices, N)

H_app_2 = tfim_perturbation.H_app_2(basis, Jij, GS_indices, N, GS_energy)

In [41]:
# Build exact matrix
V_exc = tfim_perturbation.V_exact(basis)

H_0_exc = tfim_perturbation.H_0_exact(Energies)

AttributeError: 'module' object has no attribute 'V_exact'

In [42]:
# Calculate approximated eigenvalues and eigenstates for range(h_x)
app_eigenvalues, app_eigenstates = tfim_perturbation.app_eigensystem(GS_indices, h_x_range)

AttributeError: 'module' object has no attribute 'app_eigensystem'

In [ ]:
# Calculate exact eigenvalues and eigenstates for range(h_x)
exc_eigenvalues, exc_eigenstates = tfim_perturbation.exc_eigensystem(GS_indices, h_x_range)

In [ ]:
# Plot app GS energy vs h_x
for i in range(len(GS_indices)):
    app_GS_energy_plot = plt.plot(h_x_range, app_eigenvalues[i])

plt.ylabel("Approximated GS energy eigenvalues")
plt.show(app_GS_energy_plot)

In [ ]:
# Plot exc GS energy vs h_x
for i in range(len(GS_indices)):
    exc_GS_energy_plot = plt.plot(h_x_range, exc_eigenvalues[i])

plt.ylabel("Exact GS energy eigenvalues")
plt.show(exc_GS_energy_plot)

In [ ]:
# Compare the exact and approximated GS energies

for i in range(len(GS_indices)):
    plt.plot(h_x_range, app_eigenvalues[i], h_x_range, exc_eigenvalues[i])

In [ ]:
# Calculate and plot energy errors

corrected_exc_eigenvalues = np.zeros((len(GS_indices), len(h_x_range)))

for i in range(len(GS_indices)):
    for j in range(len(h_x_range)):
        corrected_exc_eigenvalues[i, j] = exc_eigenvalues[i, j]

error_array = np.absolute(corrected_exc_eigenvalues - app_eigenvalues)

# Curve fit

for i in range(len(GS_indices)):
    popt, pcov = optimize.curve_fit(poly_3, h_x_range, error_array[i])
    GS_energy_error_plot = plt.plot(h_x_range, error_array[i])
    plt.plot(h_x_range, tfim_perturbation.poly_3(h_x_range, *popt))

In [ ]:
# Calculate probabilities for approximated eigenstates

prob_app = tfim_perturbation.prob_app(GS_indices, h_x_range, app_eigenstates)

# Plot probabilities for finding six bases states for system in ground eigenstate (approximated)

for i in range(len(GS_indices)):
    app_GS_probability_plot = plt.scatter(h_x_range, prob_app[i])

plt.ylabel("GS probability for finding given basis state in approximated Hamiltonian")
plt.show(app_GS_probability_plot)

In [ ]:
# Calculate probabilities for exact eigenstates

prob_exc = tfim_perturbation.prob_exc(GS_indices, h_x_range, exc_eigenstates)

# Plot probabilities for finding six bases states for system in ground eigenstate (exact)

for i in range(len(GS_indices)):
    exc_GS_probability_plot = plt.scatter(h_x_range, prob_exc[i])

plt.ylabel("GS probability for finding given basis state in exact Hamiltonian")
plt.show(exc_GS_probability_plot)

In [ ]:
# Compare probabilities

for i in range(len(GS_indices)):
    plt.plot(h_x_range, prob_app[i], h_x_range, prob_exc[i])

In [ ]:
# Calculate and plot probability errors

prob_error_array = np.absolute(prob_app - prob_exc)

# Curve fit
for i in range(len(GS_indices)):
    popt, pcov = optimize.curve_fit(poly_2, h_x_range, prob_error_array[i])
    GS_probability_error_plot = plt.plot(h_x_range, prob_error_array[i])
    plt.plot(h_x_range, tfim_perturbation.poly_2(h_x_range, *popt))

In [ ]:
# Probability of finding the exact system to be in excited states
prob_exc_total = tfim_perturbation.prob_exc_total(GS_indices, h_x_range, exc_eigenstates)

prob_excited_sum = tfim_perturbation.prob_excited_sum(GS_indices, h_x_range, prob_exc_total)

for i in range(len(GS_indices)):
    popt, pcov = optimize.curve_fit(poly_2, h_x_range, prob_excited_sum[i])
    plt.plot(h_x_range, prob_excited_sum[i], h_x_range, tfim_perturbation.poly_2(h_x_range, *popt))

In [ ]:
# Plot fidelity

# Extract ground states from exc_states matrix and renormalize

GS_exc_eigenstates = tfim_perturbation.GS_exc_eigenstates(GS_indices, h_x_range, exc_eigenstates)

# Identify energy levels, sort, and calculate fidelity_array to be plotted
GS_exc_eigenvalues = exc_eigenvalues[0:6]
fidelity_array = tfim_perturbation.fidelity_array(GS_indices, h_x_range, GS_exc_eigenvalues, app_eigenvalues)

# Plot fidelity array
for i in range(len(GS_indices)):
    plt.plot(h_x_range, fidelity_array[i])

In [ ]:
# Plot infidelity

infidelity_array = tfim_perturbation.infidelity_array(fidelity_array)

for i in range(len(GS_indices)):
    popt, pcov = optimize.curve_fit(poly_2, h_x_range, infidelity_array[i])
    plt.plot(h_x_range, infidelity_array[i], h_x_range, poly_2(h_x_range, *popt))